In [5]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
openai_client = OpenAI()

In [6]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

In [7]:
CalendarEvent.model_json_schema()

{'properties': {'name': {'title': 'Name', 'type': 'string'},
  'date': {'title': 'Date', 'type': 'string'},
  'participants': {'items': {'type': 'string'},
   'title': 'Participants',
   'type': 'array'}},
 'required': ['name', 'date', 'participants'],
 'title': 'CalendarEvent',
 'type': 'object'}

In [8]:
response = openai_client.responses.parse(
    model="gpt-4o-mini",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    text_format=CalendarEvent,
)

In [9]:
response

ParsedResponse[CalendarEvent](id='resp_0b24db2d18f0aa3d00699288ee94208195aa30dba56c3cea52', created_at=1771210990.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ParsedResponseOutputMessage[CalendarEvent](id='msg_0b24db2d18f0aa3d00699288ef046081958c808f2d33f29574', content=[ParsedResponseOutputText[CalendarEvent](annotations=[], text='{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}', type='output_text', logprobs=[], parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob']))], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, completed_at=1771210991.0, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, prompt_cache_retention=None, reasoning=Reasoning(effort=None, generate_

In [10]:
response.output_parsed

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

In [11]:
response.output[0].content[0].text

'{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}'

In [12]:
event = response.output_parsed

In [13]:
event

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

# Structured RAG

In [14]:
from gitsource import GithubRepositoryDataReader, chunk_documents
from minsearch import Index

reader = GithubRepositoryDataReader(
    repo_owner="evidentlyai",
    repo_name="docs",
    allowed_extensions={"md", "mdx"},
)
files = reader.read()


parsed_docs = [doc.parse() for doc in files]
chunked_docs = chunk_documents(parsed_docs, size=3000, step=1500)

index = Index(
    text_fields=["title", "description", "content"],
    keyword_fields=["filename"]
)
index.fit(chunked_docs)

print(f"Indexed {len(chunked_docs)} chunks from {len(files)} documents")


Indexed 385 chunks from 95 documents


In [15]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt, instructions)
    return answer

In [16]:
def search(query):
    results = index.search(query = query, num_results=5)
    return results
    

In [17]:
import json

instructions = """
You're a documentation assistant. Answer the QUESTION based on the CONTEXT from our documentation.

Use only facts from the CONTEXT when answering.
If the answer isn't in the CONTEXT, say so.
"""

prompt_template = """
<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(question, search_results):
    context = json.dumps(search_results, indent=2)
    return prompt_template.format(
        question=question,
        context=context
    )
    

In [18]:

def llm(user_prompt,
        instructions=None,
        model="gpt-4o-mini"
    ):
    messages = []

    if instructions:
        messages.append({
            "role": "system",
            "content": instructions
        })

    messages.append({
        "role": "user",
        "content": user_prompt
    })

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text


In [19]:
answer = rag('How do I implement llm as a judge',)

In [20]:

def llm_structured(user_prompt,
                   output_type,
                   instructions=None,
                   model="gpt-4o-mini"
    ):
    messages = []

    if instructions:
        messages.append({
            "role": "system",
            "content": instructions
        })

    messages.append({
        "role": "user",
        "content": user_prompt
    })

    response = openai_client.responses.parse(
        model=model,
        input=messages,
        text_format=output_type
    )

    return response.output_parsed


In [21]:
response = llm_structured(user_prompt="Alice and Bob are going to a Science Fair on Friday",
                          instructions="Extract the event information",
                          output_type=CalendarEvent,
)

In [22]:
response

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

In [23]:
from typing import Optional

class RAGResponse(BaseModel):
    answer: Optional[str]=None
    found_answer: bool

In [24]:
def rag_structured(query,output_type=RAGResponse):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    return llm_structured(user_prompt=prompt,
                          instructions=instructions,
                          output_type=output_type)

In [25]:
answer = rag_structured('How do I install Kafka?')

In [26]:
print(answer.answer)
print(answer.found_answer)

None
False


In [27]:
RAGResponse.model_json_schema()

{'properties': {'answer': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'default': None,
   'title': 'Answer'},
  'found_answer': {'title': 'Found Answer', 'type': 'boolean'}},
 'required': ['found_answer'],
 'title': 'RAGResponse',
 'type': 'object'}

In [28]:

instructions = """
You're a documentation assistant. Answer the QUESTION based on the CONTEXT from our documentation.

Use only facts from the CONTEXT when answering.
If the answer isn't in the CONTEXT, say so.

If you can't find the answer, set 'answer' to None
"""

In [29]:
class RAGResponse(BaseModel):
    """   
    The response from the RAG documentation system
    
    If you can't find the answer, set 'answer' to None
    """
    answer: Optional[str]=None
    found_answer: bool

In [30]:
RAGResponse.model_json_schema()

{'description': "The response from the RAG documentation system\n\nIf you can't find the answer, set 'answer' to None",
 'properties': {'answer': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'default': None,
   'title': 'Answer'},
  'found_answer': {'title': 'Found Answer', 'type': 'boolean'}},
 'required': ['found_answer'],
 'title': 'RAGResponse',
 'type': 'object'}

In [31]:
answer = rag_structured('How do I install Kafka?')
print(answer.answer)
print(answer.found_answer)

None
False


In [32]:
from pydantic import Field

class RAGResponse(BaseModel):
    answer: Optional[str]=Field(None, description="If you can't find the answer, set 'answer' to None")
    found_answer: bool =Field(description="True if the answer is found , False otherwise") 

In [33]:
RAGResponse.model_json_schema()

{'properties': {'answer': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
   'default': None,
   'description': "If you can't find the answer, set 'answer' to None",
   'title': 'Answer'},
  'found_answer': {'description': 'True if the answer is found , False otherwise',
   'title': 'Found Answer',
   'type': 'boolean'}},
 'required': ['found_answer'],
 'title': 'RAGResponse',
 'type': 'object'}

In [34]:
answer = rag_structured('How do I install Kafka?')
print(answer.answer)
print(answer.found_answer)

None
False


In [35]:
from typing import Literal

class RAGResponse(BaseModel):
    """
    This model provides a structured answer with metadata about the response,
    including confidence, categorization, and follow-up suggestions.
    """
    answer: str = Field(description="The main answer to the user's question in markdown")
    found_answer: bool = Field(description="True if relevant information was found in the documentation")
    confidence: float = Field(description="Confidence score from 0.0 to 1.0 indicating how certain the answer is")
    confidence_explanation: str = Field(description="Explanation about the confidence level")
    answer_type: Literal["how-to", "explanation", "troubleshooting", "comparison", "reference"] = Field(description="The category of the answer")
    followup_questions: list[str] = Field(description="Suggested follow-up questions the user might want to ask")

In [36]:
RAGResponse.model_json_schema()

{'description': 'This model provides a structured answer with metadata about the response,\nincluding confidence, categorization, and follow-up suggestions.',
 'properties': {'answer': {'description': "The main answer to the user's question in markdown",
   'title': 'Answer',
   'type': 'string'},
  'found_answer': {'description': 'True if relevant information was found in the documentation',
   'title': 'Found Answer',
   'type': 'boolean'},
  'confidence': {'description': 'Confidence score from 0.0 to 1.0 indicating how certain the answer is',
   'title': 'Confidence',
   'type': 'number'},
  'confidence_explanation': {'description': 'Explanation about the confidence level',
   'title': 'Confidence Explanation',
   'type': 'string'},
  'answer_type': {'description': 'The category of the answer',
   'enum': ['how-to',
    'explanation',
    'troubleshooting',
    'comparison',
    'reference'],
   'title': 'Answer Type',
   'type': 'string'},
  'followup_questions': {'description': 'S

In [37]:
answer = rag_structured('How do I install Kafka?')
print(answer.answer)
print(answer.found_answer)

None
False
